transformar stock

In [1]:
import pandas as pd
import os

# Ruta donde se encuentran los archivos CSV
ruta = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Stock Histórico\2.Histórico Stock Semanal (CSV)"
# Diccionario para guardar los dataframes con nombres únicos
dataframes = {}

# Mapeo de nombres de columnas deseadas a posibles variantes
columnas_deseadas = {
    'Material': ['Material'],
    'Centro': ['Centro'],
    'Almacén': ['Almacén'],
    'Libre utilización': ['Libre utilización'],
    'Trans./Trasl.': ['Trans./Trasl.', 'Traslado'],
    'Inspecc.de calidad': ['Inspecc.de calidad', 'Insp. Calidad']
}

# Función para determinar las columnas a usar
def columnas_a_usar(cols, mapeo):
    cols_usar = []
    for col_std, opciones in mapeo.items():
        for opcion in opciones:
            if opcion in cols:
                cols_usar.append(opcion)
                break
    return cols_usar

# Recorrer los archivos en el directorio especificado
for archivo in os.listdir(ruta):
    if archivo.endswith(".csv") and len(archivo) >= 10:
        fecha_archivo = archivo[:10]  # Extraer la fecha del nombre del archivo
        nombre_df = f"df_{archivo.replace('.csv', '')}"  # Crear un nombre único para el dataframe
        try:
            # Leer solo los nombres de las columnas primero
            cols_temp = pd.read_csv(os.path.join(ruta, archivo), delimiter=';', decimal=',', nrows=0).columns
            # Determinar qué columnas usar basadas en las columnas del archivo y las deseadas
            cols_usar = columnas_a_usar(cols_temp, columnas_deseadas)
            
            # Crear el dataframe desde el archivo CSV
            df = pd.read_csv(
                os.path.join(ruta, archivo), 
                delimiter=';', 
                decimal=',', 
                low_memory=False, 
                usecols=cols_usar
            )
            # Convertir la columna "Material" a string y eliminar ".0" al final si está presente
            df['Material'] = df['Material'].astype(str).str.replace(r'\.0$', '', regex=True)
            # Agregar la columna de fecha al dataframe
            df['Fecha'] = fecha_archivo
            
            # Limpiar y convertir la columna "Libre utilización" a entero, quitando el .0 si es necesario
            if 'Libre utilización' in df.columns:
                df['Libre utilización'] = pd.to_numeric(df['Libre utilización'], errors='coerce')  # Convertir a numérico, con valores no convertibles como NaN
                df['Libre utilización'] = df['Libre utilización'].fillna(0).astype(int)  # Reemplazar NaN con 0 y convertir a entero
            
            # Guardar el dataframe en el diccionario con un nombre único
            dataframes[nombre_df] = df
        except Exception as e:
            print(f"No se pudo leer el archivo {archivo}. Error: {e}")
            print("Es posible que alguna columna deseada no exista en este archivo.")

# Combinar todos los dataframes en un solo dataframe
combined_df = pd.concat(dataframes.values(), ignore_index=True)

# Opcional: Mostrar las primeras filas del dataframe combinado para verificar
combined_df.head(2)

,Material,Centro,Almacén,Libre utilización,Trans./Trasl.,Inspecc.de calidad,Fecha
0,1000021,710,1100.0,1,0,0.0,2022-03-28
1,1000025,710,1100.0,8,0,0.0,2022-03-28


In [2]:

suma_libre_utilizacion = combined_df[combined_df['Fecha'] == '2024-06-17']['Libre utilización'].sum()

print(f"La suma de Libre utilización para la fecha 11-06-2024 es: {suma_libre_utilizacion}")
#La suma de Libre utilización para la fecha 11-06-2024 es: 3233010

La suma de Libre utilización para la fecha 11-06-2024 es: 3233010


In [3]:
valores_unicos_fecha = combined_df['Fecha'].unique()
valores_unicos_fecha


array(['2022-03-28', '2022-04-04', '2022-04-11', '2022-04-18',
       '2022-04-25', '2022-05-02', '2022-05-09', '2022-05-16',
       '2022-05-23', '2022-05-30', '2022-06-06', '2022-06-13',
       '2022-06-20', '2022-06-28', '2022-07-04', '2022-07-11',
       '2022-07-18', '2022-07-25', '2022-08-01', '2022-08-08',
       '2022-08-16', '2022-08-22', '2022-08-29', '2022-09-05',
       '2022-09-12', '2022-09-20', '2022-09-26', '2022-10-03',
       '2022-10-11', '2022-10-17', '2022-10-24', '2022-11-02',
       '2022-11-07', '2022-11-14', '2022-11-21', '2022-11-28',
       '2022-12-06', '2022-12-13', '2022-12-20', '2022-12-27',
       '2023-01-03', '2023-01-17', '2023-01-24', '2023-01-31',
       '2023-02-06', '2023-02-13', '2023-02-20', '2023-02-27',
       '2023-03-06', '2023-03-13', '2023-03-20', '2023-03-27',
       '2023-04-03', '2023-04-10', '2023-04-17', '2023-04-24',
       '2023-05-02', '2023-05-08', '2023-05-15', '2023-05-22',
       '2023-05-29', '2023-06-05', '2023-06-12', '2023-

In [4]:
combined_df.columns

Index(['Material', 'Centro', 'Almacén', 'Libre utilización', 'Trans./Trasl.',
       'Inspecc.de calidad', 'Fecha'],
      dtype='object')

In [5]:
# # Filtramos el DataFrame por la fecha específica
# filtered_df = combined_df[combined_df['Fecha'] == '2024-06-11']
# # Sumamos la columna 'Libre utilización'
# total_libre_utilizacion = filtered_df['Libre utilización'].sum()
# total_libre_utilizacion

In [6]:
# filtered_df.head(2)

In [7]:
combined_df.shape
#(8108991, 7)

(8489415, 7)

In [8]:
combined_df.shape

(8489415, 7)

In [9]:
# Asumiendo que 'merged_df' es tu DataFrame y 'Fecha' es la columna a convertir
combined_df['Fecha'] = pd.to_datetime(combined_df['Fecha'], errors='coerce')
# Crear la columna 'Año'
combined_df['Año'] = combined_df['Fecha'].dt.year
# Crear la columna 'Mes'
combined_df['Mes'] = combined_df['Fecha'].dt.month


In [10]:
combined_df.head()

,Material,Centro,Almacén,Libre utilización,Trans./Trasl.,Inspecc.de calidad,Fecha,Año,Mes
0,1000021,710,1100.0,1,0,0.0,2022-03-28,2022,3
1,1000025,710,1100.0,8,0,0.0,2022-03-28,2022,3
2,1000026,710,1100.0,3,0,0.0,2022-03-28,2022,3
3,1000027,710,1100.0,1,0,0.0,2022-03-28,2022,3
4,1000028,710,1100.0,1,0,0.0,2022-03-28,2022,3


In [11]:
import os
import pandas as pd
import datetime
import glob

usuario = os.path.expanduser('~').split('\\')[-1]
año_actual = datetime.datetime.now().year
mes_actual = datetime.datetime.now().strftime('%m')

ruta_directorio = f"C:\\Users\\{usuario}\\DERCO CHILE REPUESTOS SpA\\Planificación y abastecimiento - Documentos\\Planificación y Compras Maestros\\{año_actual}\\{año_actual}-{mes_actual}\\MaestrosCSV\\"
archivos_cod = glob.glob(os.path.join(ruta_directorio, "*COD*.csv"))

if archivos_cod:
    archivo_cod = archivos_cod[0]
    ultimoeslabon = pd.read_csv(archivo_cod, delimiter=';', encoding='latin-1', decimal=',')
else:
    print("No se encontró ningún archivo que contenga 'COD' en el nombre en el directorio especificado.")
    

eslabon = ultimoeslabon

cruzar UE a venta historica

Venta histórica

_____AQUI TERMINA LA VENTA CON ULTIMO ESLABON Y EMPEZAMOS CON EL STOCK DENUEVO___________

In [12]:
import numpy as np

# Reemplazar los NaN por 1 en 'Almacén'
combined_df['Almacén'].fillna(1, inplace=True)

# Obtener los valores únicos de 'Almacén' y 'Centro' después de la limpieza y la conversión
valores_unicos_almacen = combined_df['Almacén'].unique()
valores_unicos_centro = combined_df['Centro'].unique()

# Imprimir los valores únicos limpios
print("Valores únicos en la columna 'Almacén' después de la limpieza:", valores_unicos_almacen)
print("Valores únicos en la columna 'Centro' después de la limpieza:", valores_unicos_centro)


C:\Users\etorres.DERCOPARTS\AppData\Local\Temp\ipykernel_9576\2443103825.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Almacén'].fillna(1, inplace=True)


Valores únicos en la columna 'Almacén' después de la limpieza: [1100.0 1600.0 1 1120.0 '1600' '1100' '    ' '1100.0' '1600.0']
Valores únicos en la columna 'Centro' después de la limpieza: [710 711 713 712 714 496 nan '714.0' '713.0' '710.0' '711.0' '712.0']


In [13]:
# Para obtener los valores únicos de la columna 'Centro'
valores_unicos_centro = combined_df['Centro'].unique()

# Para obtener los valores únicos de la columna 'Almacén'
valores_unicos_almacen = combined_df['Almacén'].unique()

# Imprimir los valores únicos
print("Valores únicos en la columna 'Centro':", valores_unicos_centro)
print("Valores únicos en la columna 'Almacén':", valores_unicos_almacen)


Valores únicos en la columna 'Centro': [710 711 713 712 714 496 nan '714.0' '713.0' '710.0' '711.0' '712.0']
Valores únicos en la columna 'Almacén': [1100.0 1600.0 1 1120.0 '1600' '1100' '    ' '1100.0' '1600.0']


Nombrar los dataframe

In [14]:
# Utilizar una expresión regular para encontrar filas donde 'Material' contiene letras
# La expresión regular '[a-zA-Z]' busca cualquier carácter que sea una letra
filas_con_letras = combined_df['Material'].astype(str).str.contains('[a-zA-Z]', regex=True)

# Eliminar las filas donde 'Material' contiene letras
combined_df = combined_df[~filas_con_letras]



In [15]:
# Reemplazar los NaN en 'Libre utilización' con 0, también usando .loc
combined_df.loc[:, 'Libre utilización'] = combined_df['Libre utilización'].fillna(0)


In [16]:
stock = combined_df
stock.head(2)


,Material,Centro,Almacén,Libre utilización,Trans./Trasl.,Inspecc.de calidad,Fecha,Año,Mes
0,1000021,710,1100.0,1,0,0.0,2022-03-28,2022,3
1,1000025,710,1100.0,8,0,0.0,2022-03-28,2022,3


Transformar Stock semanal a Mensual

In [17]:
stock.head(2)

,Material,Centro,Almacén,Libre utilización,Trans./Trasl.,Inspecc.de calidad,Fecha,Año,Mes
0,1000021,710,1100.0,1,0,0.0,2022-03-28,2022,3
1,1000025,710,1100.0,8,0,0.0,2022-03-28,2022,3


In [18]:
# import getpass
# usuario = getpass.getuser()
# año_actual = datetime.now().year
# mes_actual = datetime.now().strftime('%m')
# ruta = f"C:\\Users\\{usuario}\\DERCO CHILE REPUESTOS SpA\\Planificación y abastecimiento - Documentos\\Planificación y Compras Maestros\\{año_actual}\\{año_actual}-{mes_actual}\\MaestrosCSV\\"
# archivo_mara = None
# fecha_modificacion = None



In [19]:
import os
import pandas as pd
import datetime
import glob

usuario = os.path.expanduser('~').split('\\')[-1]
año_actual = datetime.datetime.now().year
mes_actual = datetime.datetime.now().strftime('%m')

ruta_directorio = f"C:\\Users\\{usuario}\\DERCO CHILE REPUESTOS SpA\\Planificación y abastecimiento - Documentos\\Planificación y Compras Maestros\\{año_actual}\\{año_actual}-{mes_actual}\\MaestrosCSV\\"
archivos_mara = glob.glob(os.path.join(ruta_directorio, "*MARA*.csv"))

if archivos_mara:
    archivo_mara = archivos_mara[0]
    mara = pd.read_csv(archivo_mara, delimiter=';')
else:
    print("No se encontró ningún archivo que contenga 'MARA' en el nombre en el directorio especificado.")

In [20]:

mara.rename(columns={'Material_S4': 'Material'}, inplace=True)
mara.rename(columns={'Nombre_Sector':'Sector'}, inplace=True)
stock = pd.merge(stock, mara[['Material', 'Sector']], on='Material', how='left')


In [21]:
mara

,Material,Creado_el,Sector_MU,Sector,Marca,Marca_txt,Texto_breve_material
0,159081,2019-04-02,54,RMAQ,636.0,NEXEN,CAMARA 7.50X16 VALVULA TR218A
1,1143617,2022-08-03,48,ACC,1371.0,NOMADE,CAJA PORTAEQUIPAJE PARA AUTO 450 LTS
2,1172010,2024-11-13,48,ACC,13.0,MOTORLIFE,PARAGUA AUTOPLANET TEMPORADA 2025
3,154856,2019-04-02,29,RALT,580.0,ASAHI,JG. MET. BAN MS1208A STD
4,341259,2019-02-16,48,ACC,146.0,AUTOSTYLE,AMP HB3/9005 12V65W(P20D)STD-BL1 CJ10UN
...,...,...,...,...,...,...,...
451814,1127257,2021-12-27,48,ACC,1163.0,SMART CHOICE,CAMARA DE RETROCESO SMART CHOICE 1
451815,151262,2019-02-16,54,RMAQ,100.0,AGPO,CHAPA CONTACTO C/ENCENDIDO FRI
451816,121722,2019-02-16,29,RALT,17.0,DGP,CRR AUX DGP SZ SAMURAI 1.3 87/96
451817,155909,2019-03-31,54,RMAQ,46.0,KOYO,ROD KOYO 6003 ZZ UN


In [22]:
stock.head(2)

,Material,Centro,Almacén,Libre utilización,Trans./Trasl.,Inspecc.de calidad,Fecha,Año,Mes,Sector
0,1000021,710,1100.0,1,0,0.0,2022-03-28,2022,3,Landini
1,1000025,710,1100.0,8,0,0.0,2022-03-28,2022,3,Claas


In [23]:
eslabon

,Clave PIC,NÂº sec.,Nro_pieza_fabricante_1,Cod_Actual_1,Nro_pieza_fabricante,Cod_actual,VÃ¡l.desde,Estado
0,NaN,NaN,575579,536026,NaN,NaN,NaN,NaN
1,NaN,NaN,576126,542764,NaN,NaN,NaN,NaN
2,NaN,NaN,583940,515472,NaN,NaN,NaN,NaN
3,NaN,NaN,583949,521039,NaN,NaN,NaN,NaN
4,NaN,NaN,584287,563102,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
93504,NaN,NaN,664726,1145984,NaN,NaN,NaN,NaN
93505,NaN,NaN,591321,1060424,NaN,NaN,NaN,NaN
93506,NaN,NaN,573846,505670,NaN,NaN,NaN,NaN
93507,NaN,NaN,579449,529873,NaN,NaN,NaN,NaN


In [24]:

eslabon = ultimoeslabon[['Nro_pieza_fabricante_1', 'Cod_Actual_1']].copy()
eslabon.rename(columns={'Nro_pieza_fabricante_1': 'Material'}, inplace=True)
print(eslabon.head(2))

  Material Cod_Actual_1
0   575579       536026
1   576126       542764


In [25]:
stock['Material'] = stock['Material'].astype(str)
eslabon['Material'] = eslabon['Material'].astype(str)
stock = pd.merge(stock, eslabon, left_on='Material', right_on='Material', how='left')
stock['Cod_Actual_1'].fillna(stock['Material'], inplace=True)
stock = stock.drop(['Material'], axis=1)
stock = stock.rename(columns={'Cod_Actual_1': 'material'})

C:\Users\etorres.DERCOPARTS\AppData\Local\Temp\ipykernel_9576\3144107653.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stock['Cod_Actual_1'].fillna(stock['Material'], inplace=True)


In [26]:
stock.head(2)

,Centro,Almacén,Libre utilización,Trans./Trasl.,Inspecc.de calidad,Fecha,Año,Mes,Sector,material
0,710,1100.0,1,0,0.0,2022-03-28,2022,3,Landini,1000021
1,710,1100.0,8,0,0.0,2022-03-28,2022,3,Claas,1000025


In [27]:
import pandas as pd

# Leer el archivo Excel
ruta = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras Maestros\Vigencias\Maestra Retail\Maestra Aftermarket Actualizable.xlsx"
maestra = pd.read_excel(ruta)

# Seleccionar columnas deseadas
columnas_deseadas = ['Material SAP', 'Nombre Sector', 'Agrupacion ClaCom', 'Subagrupacion ClaCom', 'Texto breve Material', 'Pais de Origen', 'Segmentacion Aftermarket', 'CES 01', 'Mayorista', 'Sodimac', 'Easy', 'Walmart', 'SMU', 'Tottus', 'Retail AP', 'Retail AG']
maestra = maestra[columnas_deseadas]

# Convertir la columna 'Material SAP' a tipo str
maestra['Material SAP'] = maestra['Material SAP'].astype(str)

# Realizar la fusión
stock = pd.merge(stock, maestra, left_on='material', right_on='Material SAP', how='left')


In [28]:
stock.columns 

Index(['Centro', 'Almacén', 'Libre utilización', 'Trans./Trasl.',
       'Inspecc.de calidad', 'Fecha', 'Año', 'Mes', 'Sector', 'material',
       'Material SAP', 'Nombre Sector', 'Agrupacion ClaCom',
       'Subagrupacion ClaCom', 'Texto breve Material', 'Pais de Origen',
       'Segmentacion Aftermarket', 'CES 01', 'Mayorista', 'Sodimac', 'Easy',
       'Walmart', 'SMU', 'Tottus', 'Retail AP', 'Retail AG'],
      dtype='object')

In [29]:
stock.shape

(8435194, 26)

In [30]:
# Filtrar el DataFrame para obtener solo las filas donde la columna 'Año' está entre 2019 y 2024 y 'Sector' está en la lista específica
sectores_deseados = ['ACC', 'LUB', 'NEU', 'BAT', 'RMAQ', 'RALT']
stock_2019_2024 = stock[stock['Sector'].isin(sectores_deseados)]

stock_2019_2024 = stock_2019_2024.copy()

stock_2019_2024.fillna(0, inplace=True)
stock_2019_2024.shape

# stock_2019_2024 = stock[(stock['Año'].between(2019, 2024)) & (stock['Sector'].isin(sectores_deseados))]
# Si stock_2019_2024 es una selección de otro DataFrame, asegúrate de hacer una copia:

(2074888, 26)

In [31]:
stock_2019_2024

,Centro,Almacén,Libre utilización,Trans./Trasl.,Inspecc.de calidad,Fecha,Año,Mes,Sector,material,...,Segmentacion Aftermarket,CES 01,Mayorista,Sodimac,Easy,Walmart,SMU,Tottus,Retail AP,Retail AG
46,710,1100.0,12,0,0.0,2022-03-28,2022,3,ACC,1000143,...,SIN SEGMENTACION,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,710,1100.0,506,0,0.0,2022-03-28,2022,3,ACC,1000144,...,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,710,1100.0,502,0,0.0,2022-03-28,2022,3,ACC,1000147,...,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
49,710,1100.0,91,0,0.0,2022-03-28,2022,3,ACC,1000148,...,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
50,710,1600.0,0,0,0.0,2022-03-28,2022,3,ACC,1000148,...,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8435189,714.0,1100.0,50,0.0,0.0,2024-12-02,2024,12,RALT,1171991,...,SIN SEGMENTACION,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
8435190,714.0,1100.0,50,0.0,0.0,2024-12-02,2024,12,RALT,1171992,...,SIN SEGMENTACION,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
8435191,714.0,1100.0,50,0.0,0.0,2024-12-02,2024,12,RALT,1171993,...,SIN SEGMENTACION,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
8435192,714.0,1100.0,100,0.0,0.0,2024-12-02,2024,12,RALT,1171994,...,SIN SEGMENTACION,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [32]:

# filas_con_letras = stock_2019_2024[stock_2019_2024['Material SAP'].str.contains('[a-zA-Z]', na=False)]

# # Obtener los valores únicos de la columna 'Material' de las filas eliminadas
# valores_unicos_material = filas_con_letras['Material SAP'].unique()

In [33]:
# valores_unicos_material

In [34]:

# Si stock_2019_2024 es una selección de otro DataFrame, asegúrate de hacer una copia
stock_2019_2024 = stock_2019_2024.copy()

# Luego realiza la operación de llenado
stock_2019_2024.fillna(0, inplace=True)

In [35]:
stock_2019_2024.tail

<bound method NDFrame.tail of         Centro Almacén  Libre utilización Trans./Trasl. Inspecc.de calidad  \
46         710  1100.0                 12             0                0.0   
47         710  1100.0                506             0                0.0   
48         710  1100.0                502             0                0.0   
49         710  1100.0                 91             0                0.0   
50         710  1600.0                  0             0                0.0   
...        ...     ...                ...           ...                ...   
8435189  714.0  1100.0                 50           0.0                0.0   
8435190  714.0  1100.0                 50           0.0                0.0   
8435191  714.0  1100.0                 50           0.0                0.0   
8435192  714.0  1100.0                100           0.0                0.0   
8435193  714.0  1100.0                  9           0.0                0.0   

             Fecha   Año  Mes Sec

In [36]:
stock_2019_2024.shape


(2074888, 26)

In [37]:
asd

NameError: name 'asd' is not defined

In [ ]:
stock_2019_2024 = stock_2019_2024[stock_2019_2024['Fecha'].dt.year > 2022]


In [ ]:
stock_2019_2024.columns

Index(['Centro', 'Almacén', 'Libre utilización', 'Trans./Trasl.',
       'Inspecc.de calidad', 'Fecha', 'Año', 'Mes', 'Sector', 'material',
       'Material SAP', 'Nombre Sector', 'Agrupacion ClaCom',
       'Subagrupacion ClaCom', 'Texto breve Material', 'Pais de Origen',
       'Segmentacion Aftermarket', 'CES 01', 'Mayorista', 'Sodimac', 'Easy',
       'Walmart', 'SMU', 'Tottus', 'Retail AP', 'Retail AG'],
      dtype='object')

In [ ]:

# # Guardar este DataFrame filtrado a un archivo CSV, delimitado por punto y coma, en la ruta especificada
# ruta = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras Ventas\stock_2019_2024.csv"  # Nota el 'r' antes de la cadena para tratar la ruta como raw string
# stock_2019_2024.to_csv(ruta, index=False, sep=';', decimal=',')

In [ ]:
# stock_2019_2024['Centro'] = stock_2019_2024['Centro'].astype(str)
# stock_2019_2024['Almacén'] = stock_2019_2024['Almacén'].astype(str)
# stock_2019_2024['Inspecc.de calidad'] = stock_2019_2024['Inspecc.de calidad'].astype(str)
# stock_2019_2024['Trans./Trasl.'] = stock_2019_2024['Trans./Trasl.'].astype(str)
# stock_2019_2024['Material SAP'] = stock_2019_2024['Material SAP'].astype(str)
# stock_2019_2024['Nombre Sector'] = stock_2019_2024['Nombre Sector'].astype(str)
# columns_to_convert = ['Centro', 'Sector', 'material', 'Material SAP', 'Nombre Sector', 
#                       'Agrupacion ClaCom', 'Subagrupacion ClaCom', 'Texto breve Material', 
#                       'Pais de Origen', 'Segmentacion Aftermarket', 'CES 01', 'Mayorista', 
#                       'Sodimac', 'Easy', 'Walmart', 'SMU', 'Tottus', 'Retail AP', 'Retail AG']

# stock_2019_2024[columns_to_convert] = stock_2019_2024[columns_to_convert].astype(str)


In [ ]:
# # parquet]
# import pandas as pd

# # Supongamos que stock_2019_2024 es tu DataFrame
# # stock_2019_2024 = pd.DataFrame(...)  # Carga o define tu DataFrame aquí

# # Ruta para guardar el archivo Parquet
# ruta = r"C:\Users\Etorres.dercoparts\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\stock_2019_2024.parquet"
# # Guardar el DataFrame en formato Parquet
# stock_2019_2024.to_parquet(ruta, index=False)


In [ ]:
# # Filtrar el DataFrame para obtener solo las filas donde la columna 'Año' está entre 2019 y 2024 y 'Sector' está en la lista específica
# sectores_deseados = ['ACC', 'LUB', 'NEU', 'BAT', 'RMAQ', 'RALT']
# stock_2019_2024 = stock[(stock['Año'].between(2023, 2023)) & (stock['Sector'].isin(sectores_deseados))]
# # Si stock_2019_2024 es una selección de otro DataFrame, asegúrate de hacer una copia:
# stock_2019_2024 = stock_2019_2024.copy()
# # Luego realiza la operación de llenado
# stock_2019_2024.fillna(0, inplace=True)

# # Guardar este DataFrame filtrado a un archivo CSV, delimitado por punto y coma, en la ruta especificada
# ruta = r"C:\Users\Etorres\Desktop\Codigos Demanda\stock2024.csv"  # Nota el 'r' antes de la cadena para tratar la ruta como raw string
# stock_2019_2024.to_csv(ruta, index=False, sep=';', decimal=',')


In [ ]:
stock_2019_2024.columns

Index(['Centro', 'Almacén', 'Libre utilización', 'Trans./Trasl.',
       'Inspecc.de calidad', 'Fecha', 'Año', 'Mes', 'Sector', 'material',
       'Material SAP', 'Nombre Sector', 'Agrupacion ClaCom',
       'Subagrupacion ClaCom', 'Texto breve Material', 'Pais de Origen',
       'Segmentacion Aftermarket', 'CES 01', 'Mayorista', 'Sodimac', 'Easy',
       'Walmart', 'SMU', 'Tottus', 'Retail (AP-AG)'],
      dtype='object')

In [ ]:
# import pandas as pd
# # Supongamos que ya tienes tu DataFrame llamado 'stock' cargado
# # Mostrar los tipos de datos de todas las columnas del DataFrame
# print(stock_2019_2024.dtypes)